In [1]:
# Import the warnings package to ignore warning messages to limit clutter in the console

import warnings
warnings.filterwarnings('ignore')

%load_ext autotime

time: 199 µs (started: 2023-06-15 06:27:32 +00:00)


In [2]:
import pandas as pd
import numpy as np
from Target_Encoder import Target_Encoder

from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict, cross_validate, RandomizedSearchCV, KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score,ConfusionMatrixDisplay, precision_recall_curve, auc, roc_curve, PrecisionRecallDisplay
from sklearn.metrics import balanced_accuracy_score,precision_score, recall_score
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeCV, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, VotingClassifier
from sklearn.inspection import permutation_importance
from sklearn.svm import SVC
from typing import Tuple

import catboost as cb
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt

time: 1.33 s (started: 2023-06-15 06:27:32 +00:00)


In [3]:
# Import data from the csv file 
# In this version of the model, we will be using all the parameters to build the model
ModelData_All = pd.read_csv('UHG_WOs.csv', engine = 'c')
ModelData_All.head()
ModelData_All.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192927 entries, 0 to 192926
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   country_code               192927 non-null  object
 1   client_property_name       192927 non-null  object
 2   workorder_number           192927 non-null  object
 3   WhoDidIt                   192927 non-null  object
 4   workorder_desc             192921 non-null  object
 5   WOCreatedDate              192927 non-null  object
 6   WODateCompleted            127076 non-null  object
 7   SLA                        192927 non-null  object
 8   priority_code              192927 non-null  object
 9   type_code                  192927 non-null  object
 10  category_code              192927 non-null  int64 
 11  category_code_desc         192927 non-null  object
 12  sub_category_code          192927 non-null  int64 
 13  sub_category_code_desc     192927 non-null  

In [4]:
ModelData_US = ModelData_All[ModelData_All['country_code'] == 'US']

time: 36.5 ms (started: 2023-06-15 06:27:34 +00:00)


In [5]:
ModelData_US.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147259 entries, 45668 to 192926
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   country_code               147259 non-null  object
 1   client_property_name       147259 non-null  object
 2   workorder_number           147259 non-null  object
 3   WhoDidIt                   147259 non-null  object
 4   workorder_desc             147256 non-null  object
 5   WOCreatedDate              147259 non-null  object
 6   WODateCompleted            93707 non-null   object
 7   SLA                        147259 non-null  object
 8   priority_code              147259 non-null  object
 9   type_code                  147259 non-null  object
 10  category_code              147259 non-null  int64 
 11  category_code_desc         147259 non-null  object
 12  sub_category_code          147259 non-null  int64 
 13  sub_category_code_desc     147259 non-nu

In [6]:
ModelData_US['SLA'].value_counts()

On time    83868
Unknown    53552
Missed      9839
Name: SLA, dtype: int64

time: 19.2 ms (started: 2023-06-15 06:27:34 +00:00)


In [7]:
83868/9839

8.524036995629638

time: 1.69 ms (started: 2023-06-15 06:27:34 +00:00)


In [8]:
ModelData_US['country_code'].value_counts() 

US    147259
Name: country_code, dtype: int64

time: 16.9 ms (started: 2023-06-15 06:27:34 +00:00)


In [108]:
ModelData_US['client_property_name'].value_counts()

WA052-Everett-3901 Hoyt Ave                   3357
MN008-Minnetonka-9900 Bren Rd East            3275
MN101-Eden Prairie-11000 Optum Cir            2519
WA061-Shoreline-1201 N 175th St               2342
WA044-Everett-3927 Rucker Ave                 2263
                                              ... 
VA117-Henrico-8215 Hermitage Rd                  1
IL154-Moline-4200 49th Ave                       1
CO224-1715 Iron Horse Dr-Ste 100, Ste 102        1
CA337-Northridge-18433 Roscoe Blvd-STE 206       1
WA030-Seattle-9706 4th Ne Ave                    1
Name: client_property_name, Length: 1657, dtype: int64

time: 16.5 ms (started: 2023-06-16 11:20:33 +00:00)


In [9]:
ModelData_US['priority_code'].value_counts()

P9     86952
P3     21441
P6     14528
P7     11871
P5      7489
P2C     3305
P2A     1132
P1C      285
R1A      140
P1A      107
P3         5
P7         2
P4         2
Name: priority_code, dtype: int64

time: 16 ms (started: 2023-06-15 06:27:34 +00:00)


In [10]:
ModelData_US['type_code'].value_counts()

PM                    88162
Corrective            53984
SelfGen                4876
Install                  84
Project                  43
FM Project Expense       37
FM Project Capex         32
Move                     28
Enhance                  12
Vendor Credit             1
Name: type_code, dtype: int64

time: 15.7 ms (started: 2023-06-15 06:27:34 +00:00)


In [11]:
ModelData_US['category_code'].value_counts()

10    89183
6     38375
5      6275
11     2892
9      2182
8      1939
2      1759
7      1643
4      1511
3      1202
13       83
16       74
17       67
15       48
14       26
Name: category_code, dtype: int64

time: 3.24 ms (started: 2023-06-15 06:27:34 +00:00)


In [12]:
ModelData_US['category_code_desc'].value_counts()

Preventive Maintenance         89183
Maintenance & Repair           38375
Janitorial                      6275
Security                        2892
Office Services                 2182
Moves/Adds/Changes              1939
Environmental/Health/Safety     1759
General Facilities Services     1643
Grounds and Parking             1511
Furniture/Fixture/Equipment     1202
Building Monitoring System        83
Permits/License/Fees              74
FM Project                        67
Lab Equipment                     48
Energy Management System          26
Name: category_code_desc, dtype: int64

time: 16.3 ms (started: 2023-06-15 06:27:34 +00:00)


In [13]:
ModelData_US['sub_category_code'].value_counts()

1     100989
17      9702
23      6201
15      6084
19      4851
5       4738
3       4437
7       3789
9       3028
21      1160
27       578
25       433
13       420
11       288
56       180
29       129
59        74
58        48
60        36
4         32
61        31
57        26
45         5
Name: sub_category_code, dtype: int64

time: 3.28 ms (started: 2023-06-15 06:27:34 +00:00)


In [14]:
ModelData_US['sub_category_code_desc'].value_counts()

Preventive Maintenance          89183
Interior Repair                  9702
Plumbing                         6201
HVAC                             6079
Cleaning                         6060
Lighting                         4851
Electrical                       3294
Doors                            2955
Lock & Key                       2753
Cafeteria/Breakroom              2358
Moves                            1939
General Facility Services        1576
Fire Suppression                 1345
Furniture                        1170
Pest Control                     1160
Conference Room                  1025
Breakroom Supplies                754
Landscaping                       613
Signs                             578
Roofing                           433
Office Services                   386
Parking Lot                       379
Snow Removal                      375
Exterior Repair                   272
Elevators                         266
Recycling & Waste                 215
Air Quality 

time: 16.7 ms (started: 2023-06-15 06:27:34 +00:00)


In [15]:
ModelData_US['repair_category_code'].value_counts()

311005    37496
311001    21871
311009     9268
311003     8883
141806     6984
          ...  
171704        1
311021        1
221007        1
221001        1
311016        1
Name: repair_category_code, Length: 178, dtype: int64

time: 3.66 ms (started: 2023-06-15 06:27:34 +00:00)


In [16]:
ModelData_US['repair_category_code_desc'].value_counts()

HVAC PM                             37496
Electrical PM                       21871
Plumbing PM                          9268
Fire/Life Safety PM                  8883
General PM                           5491
                                    ...  
FM Project CX Electrical                1
Certification at Surgery Centers        1
DVR - Repairs                           1
FM Project EX Electrical                1
Lift/Hoist - PM                         1
Name: repair_category_code_desc, Length: 182, dtype: int64

time: 17 ms (started: 2023-06-15 06:27:35 +00:00)


In [17]:
ModelData_US['problem_code_id'].value_counts()

311005    37496
311001    21871
311009     9268
311003     8883
141806     6984
          ...  
171704        1
311021        1
221007        1
221001        1
311016        1
Name: problem_code_id, Length: 178, dtype: int64

time: 3.4 ms (started: 2023-06-15 06:27:35 +00:00)


In [18]:
ModelData_US['problem_code_desc'].value_counts()

HVAC PM                             37496
Electrical PM                       21871
Plumbing PM                          9268
Fire/Life Safety PM                  8883
General PM                           5491
                                    ...  
FM Project CX Electrical                1
Certification at Surgery Centers        1
DVR - Repairs                           1
FM Project EX Electrical                1
Lift/Hoist - PM                         1
Name: problem_code_desc, Length: 182, dtype: int64

time: 18.5 ms (started: 2023-06-15 06:27:35 +00:00)


In [19]:
# Extract Time Features 

def spinup_timeFeatures(df: pd.DataFrame) -> pd.DataFrame:
    
    day_part_map = {1: 'Late Night', 2: 'Early Morning',
                    3: 'Morning', 4: 'Noon',
                    5: 'Evening', 6: 'Night'}
    
    df['WO_Time'] = pd.to_datetime(df['WOCreatedDate'])
    df['Date'] = df['WO_Time'].dt.date
    df['Date_dt'] = pd.to_datetime(df['Date'])
    df['Year'] = df.WO_Time.dt.year
    df["Month"] = df.WO_Time.dt.month_name()
    df['DayOfMonth'] = df.WO_Time.dt.day
    df['DayOfWeek'] = df.WO_Time.dt.day_name()
    df['TimePartOfDay'] = (df.WO_Time.dt.hour % 24 + 4) // 4  # to split 24 hours into 6 bins

    #df['TimePartOfDay'] = df['TimePartOfDay'].map(day_part_map)

    return df

time: 905 µs (started: 2023-06-15 06:27:35 +00:00)


In [20]:
ModelData = spinup_timeFeatures(ModelData_US)

time: 3.65 s (started: 2023-06-15 06:27:35 +00:00)


In [21]:
ModelData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147259 entries, 45668 to 192926
Data columns (total 26 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   country_code               147259 non-null  object        
 1   client_property_name       147259 non-null  object        
 2   workorder_number           147259 non-null  object        
 3   WhoDidIt                   147259 non-null  object        
 4   workorder_desc             147256 non-null  object        
 5   WOCreatedDate              147259 non-null  object        
 6   WODateCompleted            93707 non-null   object        
 7   SLA                        147259 non-null  object        
 8   priority_code              147259 non-null  object        
 9   type_code                  147259 non-null  object        
 10  category_code              147259 non-null  int64         
 11  category_code_desc         147259 non-null  obje

In [22]:
ModelData.head()

,country_code,client_property_name,workorder_number,WhoDidIt,workorder_desc,WOCreatedDate,WODateCompleted,SLA,priority_code,type_code,...,problem_code_id,problem_code_desc,WO_Time,Date,Date_dt,Year,Month,DayOfMonth,DayOfWeek,TimePartOfDay
45668,US,AL011-Mobile-917 Western America Cir-The Execu...,P24843,Selfgen,"Switchboard, <600V-A-CBRE-Global-EN",14-10-2022 07:03,NaN,Unknown,P9,PM,...,311001,Electrical PM,2022-10-14 07:03:00,2022-10-14,2022-10-14,2022,October,14,Friday,2
45669,US,AL011-Mobile-917 Western America Cir-The Execu...,P6663,Selfgen,"Lighting, Emergency, Battery-M-CBRE-Global-EN",01-10-2022 07:02,NaN,Unknown,P9,PM,...,311001,Electrical PM,2022-01-10 07:02:00,2022-01-10,2022-01-10,2022,January,10,Monday,2
45670,US,AL012-Birmingham-1100 Lee Branch Ln,C1020,Selfgen,Empty Trash can:>trash in my office is overflo...,26-09-2022 18:30,NaN,Unknown,P6,Corrective,...,131119,Empty Trash can,2022-09-26 18:30:00,2022-09-26,2022-09-26,2022,September,26,Monday,5
45671,US,AL012-Birmingham-1100 Lee Branch Ln,C1022,Selfgen,Exterior Lights Out Repair:>light out above do...,26-09-2022 19:50,NaN,Unknown,P3,Corrective,...,141902,Exterior Lights Out Repair,2022-09-26 19:50:00,2022-09-26,2022-09-26,2022,September,26,Monday,5
45672,US,AL012-Birmingham-1100 Lee Branch Ln,C114222,Vendor,Electrical - Repair or Replace:>ATTN TARA:\nPl...,20-02-2023 22:35,22-02-2023 22:00,On time,P5,Corrective,...,141302,Electrical - Repair or Replace,2023-02-20 22:35:00,2023-02-20,2023-02-20,2023,February,20,Monday,6


time: 22 ms (started: 2023-06-15 06:27:38 +00:00)


In [23]:
ModelData['Year'].value_counts()

2023    93028
2022    54231
Name: Year, dtype: int64

time: 3.28 ms (started: 2023-06-15 06:27:38 +00:00)


In [24]:
ModelData = ModelData[(ModelData['Date_dt'] < "2023-03-01")]

time: 94.4 ms (started: 2023-06-15 06:27:38 +00:00)


In [25]:
ModelData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119812 entries, 45668 to 192921
Data columns (total 26 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   country_code               119812 non-null  object        
 1   client_property_name       119812 non-null  object        
 2   workorder_number           119812 non-null  object        
 3   WhoDidIt                   119812 non-null  object        
 4   workorder_desc             119809 non-null  object        
 5   WOCreatedDate              119812 non-null  object        
 6   WODateCompleted            71101 non-null   object        
 7   SLA                        119812 non-null  object        
 8   priority_code              119812 non-null  object        
 9   type_code                  119812 non-null  object        
 10  category_code              119812 non-null  int64         
 11  category_code_desc         119812 non-null  obje

In [26]:
#falkflsdkfjlkdjfsdkjf

time: 225 µs (started: 2023-06-15 06:27:39 +00:00)


In [27]:
UnknownData = ModelData[(ModelData.SLA == 'Unknown')]

time: 19.3 ms (started: 2023-06-15 06:27:39 +00:00)


In [28]:
len(UnknownData)

48711

time: 1.86 ms (started: 2023-06-15 06:27:39 +00:00)


In [29]:
Model_Data = ModelData[(ModelData.SLA != 'Unknown')]

time: 25.6 ms (started: 2023-06-15 06:27:39 +00:00)


In [30]:
len(Model_Data)

71101

time: 1.71 ms (started: 2023-06-15 06:27:39 +00:00)


In [31]:
127076+65851

192927

time: 1.74 ms (started: 2023-06-15 06:27:39 +00:00)


In [32]:
Model_Data['Target_Label'] = np.where(Model_Data['SLA'] == 'Missed', 1 ,0)
Model_Data['Target_Label'].value_counts()

0    63024
1     8077
Name: Target_Label, dtype: int64

time: 8.04 ms (started: 2023-06-15 06:27:39 +00:00)


In [33]:
63024/8077

7.802897115265569

time: 1.6 ms (started: 2023-06-15 06:27:39 +00:00)


In [34]:
Model_Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71101 entries, 45672 to 192921
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   country_code               71101 non-null  object        
 1   client_property_name       71101 non-null  object        
 2   workorder_number           71101 non-null  object        
 3   WhoDidIt                   71101 non-null  object        
 4   workorder_desc             71099 non-null  object        
 5   WOCreatedDate              71101 non-null  object        
 6   WODateCompleted            71101 non-null  object        
 7   SLA                        71101 non-null  object        
 8   priority_code              71101 non-null  object        
 9   type_code                  71101 non-null  object        
 10  category_code              71101 non-null  int64         
 11  category_code_desc         71101 non-null  object        
 12 

## Model Data

In [35]:
# Global Variables
KEY_COLS = ['client_property_name','WhoDidIt','priority_code','type_code','category_code','sub_category_code','problem_code_id',
            'Month','DayOfWeek','TimePartOfDay','Target_Label']
CATEGORICALS = ['client_property_name','WhoDidIt','priority_code','type_code','Month','DayOfWeek']
NUMERICS = ['category_code','sub_category_code','problem_code_id','TimePartOfDay']
TARGET = 'Target_Label'

time: 534 µs (started: 2023-06-15 06:27:39 +00:00)


In [36]:
UseData = Model_Data[KEY_COLS]
UseData = UseData.drop_duplicates()

time: 46.1 ms (started: 2023-06-15 06:27:39 +00:00)


In [37]:
UseData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39194 entries, 45672 to 192921
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   client_property_name  39194 non-null  object
 1   WhoDidIt              39194 non-null  object
 2   priority_code         39194 non-null  object
 3   type_code             39194 non-null  object
 4   category_code         39194 non-null  int64 
 5   sub_category_code     39194 non-null  int64 
 6   problem_code_id       39194 non-null  int64 
 7   Month                 39194 non-null  object
 8   DayOfWeek             39194 non-null  object
 9   TimePartOfDay         39194 non-null  int64 
 10  Target_Label          39194 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 3.6+ MB
time: 16.8 ms (started: 2023-06-15 06:27:39 +00:00)


In [38]:
UseData['Target_Label'].value_counts()

0    32685
1     6509
Name: Target_Label, dtype: int64

time: 2.91 ms (started: 2023-06-15 06:27:39 +00:00)


In [39]:
32685/6509

5.0215086802888305

time: 1.96 ms (started: 2023-06-15 06:27:39 +00:00)


In [40]:
CATEGORICALS

['client_property_name',
 'WhoDidIt',
 'priority_code',
 'type_code',
 'Month',
 'DayOfWeek']

time: 1.96 ms (started: 2023-06-15 06:27:39 +00:00)
